## EXploring

In [1]:
import requests
import os
import re
import logging
import textwrap
import matplotlib.pyplot as plt
import pandas as pd
from googleapiclient.discovery import build
from transformers import pipeline
# from dotenv import load_dotenv

from IPython.display import JSON

In [2]:
from google.colab import userdata
yt_api = userdata.get('youtube_api')

In [3]:
api_service_name = "youtube"
api_version = "v3"

youtube = build(api_service_name, api_version, developerKey = yt_api)
request = youtube.channels().list(part="snippet,contentDetails,statistics",id="UC_x5XG1OV2P6uZZ5FSM9Ttw")

response = request.execute()
JSON(response)

<IPython.core.display.JSON object>

In [4]:
channel_id = 'UC_x5XG1OV2P6uZZ5FSM9Ttw'
url = f'https://www.googleapis.com/youtube/v3/channels?part=snippet&id={channel_id}&key={yt_api}'

response = requests.get(url)
data = response.json()

In [5]:
if 'items' in data and len(data['items']) > 0:
    channel_name = data['items'][0]['snippet']['title']
    print(f"Channel Name: {channel_name}")
else:
    print("Channel not found or invalid API key.")

Channel Name: Google for Developers


In [ ]:
# to extract the videos

In [8]:
channel_id = 'UC_x5XG1OV2P6uZZ5FSM9Ttw'
channel_url = f'https://www.googleapis.com/youtube/v3/channels?part=contentDetails&id={channel_id}&key={yt_api}'

channel_res = requests.get(channel_url).json()
uploads_playlist_id = channel_res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

videos_url = f'https://www.googleapis.com/youtube/v3/playlistItems?part=snippet&playlistId={uploads_playlist_id}&maxResults=2&key={yt_api}'
videos_res = requests.get(videos_url).json()

for item in videos_res['items']:
    video_title = item['snippet']['title']
    video_id = item['snippet']['resourceId']['videoId']
    video_url = f"https://www.youtube.com/watch?v={video_id}"
    print(f"Title: {video_title}")
    print(f"URL: {video_url}")
    print("-" * 50)

Title: AI-powered patient engagement - CareMessage and the Gemini API
URL: https://www.youtube.com/watch?v=EysJTNLQVZw
--------------------------------------------------
Title: The #GoogleIO recap playlist is live. ✨
URL: https://www.youtube.com/watch?v=B8Fdxdri0aQ
--------------------------------------------------


In [9]:
video_id

'B8Fdxdri0aQ'

In [10]:
# extracting video id from video url

In [11]:
def extract_video_id(url):
    match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
    return match.group(1) if match else None

In [12]:
url = 'https://www.youtube.com/watch?v=nrakQhg_lf8'
video_id = extract_video_id(url)
video_id

'nrakQhg_lf8'

In [13]:
# to extract comments from the video

In [15]:
video_id = 'D56_Cx36oGY'
comments_url = f"https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId={video_id}&maxResults=5&key={yt_api}"

response = requests.get(comments_url)
data = response.json()

print(f"Comments for Video ID: {video_id}\n")

if "items" in data:
    for i, item in enumerate(data["items"], 1):
        comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        author = item["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
        print(f"{i}. {author}: {comment}\n")
else:
    print("No comments found or the video might have comments disabled.")

Comments for Video ID: D56_Cx36oGY

1. @doodlelifebyTammy: i can&#39;t stress how thankful I am to have found your channel! You Explained everything so well!❤❤

2. @naveethsuraj6214: is it ok to push our code with our API key to GIT?? or we need to make some change.

3. @CultureofSpeech: Greetings 🤗 how is you data analysis going on now?

4. @shanthankasula5503: If you get an error at <a href="https://www.youtube.com/watch?v=D56_Cx36oGY&amp;t=920">15:20</a>...that&#39;s because we need to import parse library..<br><br>here is the code:<br><br>from dateutil import parser

5. @shanthankasula5503: How did you get the id in video statistics? @<a href="https://www.youtube.com/watch?v=D56_Cx36oGY&amp;t=699">11:39</a>



## **Project**

In [16]:
from google.colab import userdata
yt_api = userdata.get('youtube_api')

In [17]:
# Load Sentiment Model
sentiment_classifier = pipeline(model= "lxyuan/distilbert-base-multilingual-cased-sentiments-student", top_k=None)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def extract_video_id(url):
  patterns = [
    r"(?:https?:\/\/)?(?:www\.)?youtube\.com\/watch\?v=([^&]+)",
    r"(?:https?:\/\/)?(?:www\.)?youtube\.com\/embed\/([^?]+)",
    r"(?:https?:\/\/)?(?:www\.)?youtube\.com\/v\/([^?]+)",
    r"(?:https?:\/\/)?youtu\.be\/([^?]+)"]

  for pattern in patterns:
    match = re.search(pattern, url)
    if match:
      return match.group(1)
  return None

In [ ]:
# https://www.youtube.com/watch?v=D56_Cx36oGY
extract_video_id('https://youtu.be/2HfSFdPEFRg?si=A0ksB3BWN6IlDCex')

'2HfSFdPEFRg'

In [ ]:
def get_video_title(video_id):
  youtube = build("youtube", "v3", developerKey=yt_api)
  try:
    request = youtube.videos().list(part="snippet", id=video_id)
    response = request.execute()
    return response["items"][0]["snippet"]["title"]
  except:
    return "Unknown Title"

In [ ]:
get_video_title('2HfSFdPEFRg')

'How to use a Public API | Using a Public API with Python'

In [ ]:
def get_comments(video_id, max_results= 2):
  youtube = build("youtube", "v3", developerKey=yt_api)
  comments = []
  next_page_token = None
  try:
    while len(comments) < max_results:
        request = youtube.commentThreads().list(
          part="snippet",
          videoId=video_id,
          maxResults=min(100, max_results - len(comments)),
          textFormat="plainText",
          pageToken=next_page_token
        )
        response = request.execute()
        for item in response.get("items", []):
          comments.append(item["snippet"]["topLevelComment"]["snippet"]["textDisplay"])
        next_page_token = response.get("nextPageToken")
        if not next_page_token:
          break
  except Exception as e:
      return [], str(e)
  return comments, None

In [ ]:
video_id = '2HfSFdPEFRg'
comments = get_comments(video_id)
comments

(["I love the issue @Alex ran into with the notebook yelling at him for having too great a data rate. So real. Nothing ever goes smoothly (give the Sp*tify or T*cketmaster API a go), and it was nice to see he's human.  Great video!",
  'thank you so much man, im learning so much from this journey'],
 None)

In [ ]:
comments[0]

["I love the issue @Alex ran into with the notebook yelling at him for having too great a data rate. So real. Nothing ever goes smoothly (give the Sp*tify or T*cketmaster API a go), and it was nice to see he's human.  Great video!",
 'thank you so much man, im learning so much from this journey']

In [ ]:
def analyze_sentiment(comments):
  results = []
  counts = {"positive": 0, "neutral": 0, "negative": 0}
  all_sentiments = sentiment_classifier(comments, batch_size=8)
  for comment, scores in zip(comments, all_sentiments):
    sentiment = max(scores, key=lambda x: x["score"])
    label = sentiment["label"]
    results.append({"Comment": comment, "Sentiment": label, "Score": sentiment["score"]})
    counts[label] += 1
  return results, counts

In [ ]:
analyze_sentiment(comments[0])

([{'Comment': "I love the issue @Alex ran into with the notebook yelling at him for having too great a data rate. So real. Nothing ever goes smoothly (give the Sp*tify or T*cketmaster API a go), and it was nice to see he's human.  Great video!",
   'Sentiment': 'positive',
   'Score': 0.8595536351203918},
  {'Comment': 'thank you so much man, im learning so much from this journey',
   'Sentiment': 'positive',
   'Score': 0.875322163105011}],
 {'positive': 2, 'neutral': 0, 'negative': 0})